<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/jina/22020314_sub4_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys
import pandas as pd
import numpy as np 
import re
from typing import List, Dict
import math, copy, time
import random

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [12]:
df=pd.read_csv('sports_news_data.csv', header=0)
df.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


In [13]:
def clean_text(texts):
    corpus = [] 
    for i in range(0, len(texts)):
        review = re.sub(r'\s+', ' ', str(texts[i]))  #과다한 띄어쓰기를 한번의 띄어쓰기로 변경
        review = review.lower() #영어 일단 소문자화
        review = re.sub(r'\d{2,3}\) \d{3,4}-\d{4}','',review) #전화번호 제거1
        review = re.sub(r'\d{2,3}-\d{3,4}-\d{4}','',review) #전화번호 제거2
        review = re.sub(r'[a-zA-Z0-9_-]+@[a-z]+.[a-z]+','',review) #이메일 제거
        review = re.sub(r'사진.*','',review) #사진 출처 제거
        review = re.sub(r'\[스포탈코리아.*\]','',review) #기사 출처 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'[^ A-Za-z0-9가-힣+]',' ',review) #특수문자 제거
        review = re.sub(r'\s+', ' ', review) #과다한 띄어쓰기를 한번의 띄어쓰기로 변경22
        review = re.sub(r"^\s+", '', review) #띄어쓰기로 시작하는 것 제거
        review = re.sub(r'\s+$', '', review) #띄어쓰기로 끝나는 것 제거
        corpus.append(review)
    return corpus
#review = re.sub(r'[-=+,#/\?:■^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', review) #remove punctuation 

In [14]:
df['CONTENT']=clean_text(df.CONTENT.values.tolist())

In [15]:
df.isnull().sum()

TITLE         0
CONTENT       0
PUBLISH_DT    0
dtype: int64

In [17]:
df['TITLE']=clean_text(df.TITLE.values.tolist())

In [18]:
df.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹 uefa 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전...,2021-07-15
1,디 마리아 없다 유로x코파 베스트11 이탈리아만 7명,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까 지난달...,2021-07-15
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다 영국 매체 스포츠 바이...,2021-07-15
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다 이재성은 20...,2021-07-15
4,홈킷과 딴판 바르사 팬들 new 어웨이 셔츠 호평 가장 좋아하는 색,fc 바르셀로나가 새 시즌 원정 유니폼을 공개했다 팬들은 만족스럽다는 반응이다 바르...,2021-07-15


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9077 entries, 0 to 9076
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TITLE       9077 non-null   object
 1   CONTENT     9077 non-null   object
 2   PUBLISH_DT  9077 non-null   object
dtypes: object(3)
memory usage: 212.9+ KB
